- CARTESI/O CONFIG FILE:
========================
Here we are going to compile a file to specify solver and control problem (Inverse Kinematics) for the CartesI/O framework.
The data specified will be used to instantiate ```Tasks```, ```Constraints``` and ```Solver``` from OpenSoT as well as ```Interactive Markers``` and ROS services and topics.

Solver Options:
----------------------
The first thing to include are related to the back-end used by the front-end which is ```iHQP/LP```:
```
solver_options:
    regularization: 1e-6
    back_end: "osqp"
```
```regularization``` is a constant used for the solver regularization term, ```back_end``` specify the particular solver used, in our case ```osqp``` (you can try to use ```qpoases``` as well).

IK Problem Specification:
------------------------------------
First we decide which tasks we want to solve and which priorities, for example:
```
stack:
    - ["LFoot","RFoot","HeadPostural"]
    - ["CoMXY","WaistYaw"]
    - ["LArm","RArm"]
    - ["BodyPostural"]
```
here we have at the first level: ```LFoot```,```RFoot``` and ```HeadPostural``` which are all at the same first priority. The first two are full 6DoFs Cartesian tasks while the latter in a postural task to move the joints of the head. At second priority there are ```CoMXY``` and ```WaistYaw```; in this case we are interested to control 2 Cartesian directions for the CoM task and only the Yaw rotation for the Waist. Third priority we have both arms, 6DoFs Cartesian tasks and finally, last priority a postural. 
For what concern constraints, we want to consider joint limits and joint velocity limits, this can be done as:
```
constraints: ["JointLimits", "VelocityLimits"]
```
in particular, these constraints will act to all the levels of the stack.

Now we have to specify each task, for example the ```LFoot``` will be:
```
LFoot:
    type: "Cartesian"
    distal_link: "left_sole_link"
    base_link: "world"
```
in this way we specify that ```LFoot``` is a full 6DoFs ```Cartesian``` task defined between ```world``` frame and ```left_sole_link```.

The task on the ```CoMXY``` will be defined as:
```
CoMXY:
    type: "Com"
    lambda: 0.1
    indices: [0,1]
```
notice that the CoM is always specified in ```world``` frame. ```lambda``` is a a gain related to the position feedback of the task and ```indices``` is used to specify that only the X and Y directions (```0``` and ```1```) are controlled. Similarly for the ```WaistYaw``` we will have:
```
WaistYaw:
    type: "Cartesian"
    distal_link: "torso_2_link"
    base_link: "world"
    lambda: 0.1
    indices: [5]
```

The final file will be:
```
stack:
    - ["LFoot", "RFoot", "HeadPostural"]
    - ["CoMXY", "WaistYaw"]
    - ["LArm", "RArm"]
    - ["BodyPostural"]
    
constraints: ["JointLimits", "VelocityLimits"]


################################################
############### TASK DEFINITIONS ###############
################################################

LFoot:
    type: "Cartesian"
    distal_link: "left_sole_link"
    base_link: "world"

RFoot:
    type: "Cartesian"
    distal_link: "right_sole_link"
    base_link: "world"

LArm:
    type: "Cartesian"
    distal_link: "hand_left_palm_link"
    base_link: "world"
    lambda: 0.1

RArm:
    type: "Cartesian"
    distal_link: "hand_right_palm_link"
    base_link: "world"
    lambda: 0.1

WaistYaw:
    type: "Cartesian"
    distal_link: "torso_2_link"
    base_link: "world"
    lambda: 0.1
    indices: [5]


CoMXY:
    type: "Com"
    lambda: 0.1
    indices: [0,1]
    
BodyPostural:
    type: "Postural"
    lambda: 0.1

HeadPostural:
    type: "Postural"
    indices: [34, 35]
    lambda: 0.1
```